In [ ]:
import os
import requests
import sys

import azureml.core
from azureml.core import Experiment, Workspace
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import ProjectSystemException, UserErrorException





In [ ]:
# Check core SDK version number
print(f"Azure ML SDK Version: {azureml.core.VERSION}")

In [ ]:
# Let's define these variables here - These pieces of information can be found on the portal
subscription_id = os.getenv("SUBSCRIPTION_ID", default="<our_subscription_id>")
resource_group =  os.getenv("RESOURCE_GROUP", default="<our_resource_group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<our_workspace_name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="<our_workspace_region>")

try:
    # Let's load the workspace from the configuration file
    ws = Workspace.from_config()
    print("Workspace was loaded successfully from the configuration file")
except (UserErrorException, ProjectSystemException):
    # or directly from Azure, if it already exists (exist_ok=True).
    # If it does not exist, let's create a workspace from scratch
    ws = Workspace.create(name=workspace_name,
                          subscription_id=subscription_id,
                          resource_group=resource_group,
                          location=workspace_region,
                          create_resource_group=True,
                          exist_ok=True
                         )
    ws.write_config()
    print("Workspace was loaded successfully from Azure")

In [ ]:
# Print the workspace attributes
print(f'Workspace name: {ws.name}\n \
      Azure region: {ws.location}\n \
      Subscription id: {ws.subscription_id}\n \
      Resource group: {ws.resource_group}')

In [ ]:
MODEL_NAME = 'infersent1'
PICKLED_MODEL_NAME = MODEL_NAME + '.pkl' #assumption - it's on local

In [ ]:
model = Model.register(
    model_path = os.path.join("C:/NLP/scenarios/sentence_similarity/04-deploy", PICKLED_MODEL_NAME),
    model_name = PICKLED_MODEL_NAME,
    tags = {"Model": "InferSent"},
    description = "InferSent pretrained model",
    workspace = ws
)

In [ ]:
%%writefile score.py
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    # note here "infersent1.pkl" is the name of the model registered under
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('infersent1.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring -- ?????
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['torch'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with infersent model",
                                                  tags = {'area': "nlp", 'type': "sentence_similarity"}
                                                 )

image = ContainerImage.create(name = "infersent_image",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

In [ ]:
# create aks cluser

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'nlp-sentence-similarity-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
# deploy image as web service

%%time
aks_service_name ='nlp-aks-service-1'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
# test service

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()